
# Introduzione alle CNN con PyTorch


## Obiettivi didattici:  
1. Capire cos'è una **convoluzione** (filtri, kernel, stride, padding) e perché è utile con le immagini.  
2. Comprendere il **pooling** e il concetto di invarianza locale.  
3. Costruire, addestrare, valutare e salvare una piccola **CNN** in PyTorch sul dataset **MNIST**.  
4. Usare strumenti pratici: scelta automatica del **device** (CPU/GPU), barra di progresso con `tqdm`, funzione di **accuratezza**.  
5. Migliorare la rete con **regolarizzazione** (dropout, L2/weight decay), **Batch Normalization** e **scheduler** del learning rate.  
6. (Opzionale) Intuire cos’è il **transfer learning** con un modello pre-addestrato.

## Architettura CNN

![Alt text](https://media.datacamp.com/cms/ad_4nxct55fjxboktz5ezpyzmmkc28dy6tk3s_djp9uljfjwigsm4oagqnrvbr-edpro2ggylzl4odhtbc3xapxf-y527snl-i_noynj1uteapbm_erw-hijzkvaqmt9oiap8__pp083.png)






## Prerequisiti e setup

Esegui la cella seguente per importare le librerie. In **Colab** molte sono già presenti.  
Su macchina locale, ti consigliamo un ambiente virtuale e:
```bash
# (Facoltativo) Nuovo ambiente virtuale
# python -m venv .venv && source .venv/bin/activate  # Linux/Mac
# .venv\Scripts\activate                            # Windows

# Installazione pacchetti principali
pip install --upgrade torch torchvision torchaudio tqdm matplotlib scikit-learn
```


In [ ]:
! pip install --upgrade torch torchvision torchaudio tqdm matplotlib scikit-learn

In [ ]:
# Se hai bisogno di installare le librerie direttamente dal notebook, decommenta la riga seguente.
# È utile in Google Colab o in un nuovo ambiente di sviluppo.
# %pip install --upgrade torch torchvision torchaudio tqdm matplotlib scikit-learn


# Librerie standard Python
import os          # Gestione del file system (path, directory, variabili d'ambiente, ecc.)
import random      # Generazione di numeri casuali nativi di Python (utile per la riproducibilità)
import math        # Funzioni matematiche di base (es. sqrt, ceil, floor, log, ecc.)
import time        # Misura o gestione dei tempi di esecuzione del codice (utile per benchmark)


# Libreria per calcolo numerico e manipolazione di array multidimensionali
import numpy as np  # NumPy è spesso usato insieme a PyTorch per analisi e conversioni (tensor <-> array)


# Libreria per grafici e visualizzazioni (curve di training, immagini, confusion matrix, ecc.)
import matplotlib.pyplot as plt


# Libreria principale per il Deep Learning
import torch  # PyTorch: gestisce tensori, GPU, e calcolo automatico delle derivate (autograd)


# Moduli principali di PyTorch
from torch import nn, optim
# nn  → contiene layer predefiniti (Linear, Conv2d, ReLU, Dropout, ecc.) per costruire reti neurali
# optim → contiene algoritmi di ottimizzazione (Adam, SGD, RMSprop, ecc.) per aggiornare i pesi


# Sotto-modulo con funzioni matematiche/attivazioni da usare direttamente (es. F.relu, F.softmax)
import torch.nn.functional as F


# Modulo per la gestione dei dataset
from torch.utils.data import DataLoader, random_split, Subset
# DataLoader → crea batch di dati per l'addestramento
# random_split → divide un dataset in parti (es. train/validation)
# Subset → permette di selezionare sottoinsiemi specifici di un dataset


# Libreria PyTorch per la Computer Vision (dataset, trasformazioni e modelli pre-addestrati)
import torchvision
from torchvision import datasets, transforms
# datasets  → contiene dataset già pronti (MNIST, CIFAR10, ImageNet, ecc.)
# transforms → contiene trasformazioni (es. ToTensor, Normalize, Resize) per preprocessare immagini


# Libreria per barre di avanzamento eleganti e automatiche
from tqdm.auto import tqdm


# Stampa le versioni di PyTorch e Torchvision (utile per debug o compatibilità)
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)


# Funzione per fissare i semi casuali (reproducibilità)
# Garantisce che ogni esecuzione produca gli stessi risultati
def set_seed(seed: int = 42):
    random.seed(seed)                      # Fissa la casualità del modulo random di Python
    np.random.seed(seed)                   # Fissa la casualità di NumPy
    torch.manual_seed(seed)                # Fissa la casualità di PyTorch (CPU)
    torch.cuda.manual_seed_all(seed)       # Fissa la casualità di PyTorch per tutte le GPU
    torch.backends.cudnn.deterministic = True  # Imposta le operazioni cuDNN come deterministiche
    torch.backends.cudnn.benchmark = False     # Disattiva l’ottimizzazione dinamica di cuDNN (serve per coerenza nei risultati)


# Richiama la funzione di fissaggio del seme casuale
set_seed(42)


# Se è disponibile una GPU CUDA, la utilizza; altrimenti usa la CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Stampa quale dispositivo è stato selezionato
print("Device:", device)


# 🧠 Basi delle CNN — Convoluzione (Filtri, Stride, Padding, Feature Map)

Le Convolutional Neural Networks (CNN) sono reti progettate per analizzare immagini e dati spaziali, sfruttando il concetto di convoluzione: un’operazione matematica che combina un’immagine d’ingresso con un piccolo filtro (o kernel) per estrarre pattern locali come bordi, angoli o texture.

## 🔹 1. Filtri (Kernels)

Un filtro (o kernel) è una piccola matrice di pesi (es. 3×3 o 5×5) che scorre sull’immagine originale (input).
A ogni posizione, il filtro e la finestra dell’immagine vengono moltiplicati elemento per elemento e poi sommati → ottenendo un singolo numero nel risultato.

**Questo processo si ripete su tutta l’immagine, generando una feature map.**

📘 Ogni filtro impara a riconoscere un tipo specifico di pattern:

* Un filtro può riconoscere i bordi orizzontali,

* Un altro può riconoscere i bordi verticali,

* Altri possono individuare curve o texture più complesse.

📊 Se applichi 8 filtri diversi, otterrai 8 feature map, ciascuna con informazioni differenti.

📷 Esempio visivo: come 2 filtri 3×3 generano 2 feature map da un’immagine 7×7×3

![Alt text](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*IGfVdsOnPl6MIwMO4V33IQ.png)

## 🔹 2. Stride

Lo stride indica di quanti pixel il filtro avanza a ogni passo (sia orizzontale che verticale).

* Stride = 1 → il filtro si muove di 1 pixel per volta → output grande (massimo dettaglio).

* Stride = 2 → il filtro si muove di 2 pixel per volta → output più piccolo (riduzione dimensionale).

📷 Esempio visivo: kernel 2×2 con stride=2 su immagine 5×5
![Alt text](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*DYi9mm55THoCH_-vqWXd7A.png)

## 🔹 3. Padding

Il padding aggiunge dei “pixel fittizi” (spesso zero) intorno all’immagine di input per controllare la dimensione dell’output e preservare le informazioni ai bordi.

* Senza padding: l’immagine si “rimpicciolisce” dopo ogni convoluzione.

* Con padding: possiamo mantenere la stessa dimensione dell’immagine originale.

📷 Esempio visivo: aggiunta di padding=1 a un’immagine 3×3
![Alt text](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*mf0nv_ajmLn7-TdmdnxFlQ.png)

## 🔹 4. Feature Map

Il risultato della convoluzione è una nuova immagine detta feature map o activation map.
Ogni feature map rappresenta come un determinato filtro “vede” l’immagine originale.

📷 Esempio visivo: diverse feature map prodotte da filtri differenti su immagini di cifre MNIST
![Alt text](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*11AOltIb4osdgt8Dtc98Kw.png)

In [ ]:
# Creiamo una piccola immagine 2D di dimensione 6x6 inizialmente tutta a zero (pixel neri)
img = np.zeros((6, 6), dtype=np.float32)

# Impostiamo a 1.0 (pixel bianchi) un quadrato 2x2 al centro: righe 2-3 e colonne 2-3 (lo slice finale è esclusivo)
# Così otteniamo un pattern semplice su cui "vedere" l'effetto della convoluzione
img[2:4, 2:4] = 1.0  # piccolo quadrato bianco

# Definiamo un kernel (filtro) 3x3 di tipo "Sobel" per il gradiente ORIZZONTALE o VERTICALE?
# ATTENZIONE: questi pesi corrispondono al Sobel Gx (o a un suo segno invertito),
# cioè rilevano il GRADIENTE LUNGO X → evidenziano BORDI VERTICALI (cambiamenti sinistra↔destra).
# Per bordi verticali si usano valori con colonne [+1, 0, -1] (righe pesate 1,2,1).
# Intuizione: la colonna di sinistra è positiva (+1, +2, +1), quella di destra negativa (-1, -2, -1).
# Se a sinistra della finestra ho intensità più alte che a destra, la somma sarà positiva (bordi "in un verso");
# invertendo il segno si ribalta la direzione del contrasto (chiaro→scuro vs scuro→chiaro).
kernel = np.array([[ 1,  0, -1],
                   [ 2,  0, -2],
                   [ 1,  0, -1]], dtype=np.float32)

# Se volessi invece un kernel per BORDI ORIZZONTALI (gradiente lungo Y), useresti il Sobel Gy:
# kernel = np.array([[ 1,  2,  1],
#                    [ 0,  0,  0],
#                    [-1, -2, -1]], dtype=np.float32)

def conv2d_naive(image, kernel, stride=1, padding=0):
    """
    Implementazione "naive" della convoluzione 2D usando NumPy.
    NOTA: come nella maggior parte dei framework deep learning,
    qui eseguiamo in realtà una CORRELAZIONE (non ruotiamo il kernel di 180°).
    Questo è ciò che fanno anche PyTorch e TensorFlow nelle conv2d.
    """
    # Applichiamo il padding (se richiesto) aggiungendo bordi di zeri attorno all'immagine.
    # Il padding è utile per: (1) non "perdere" informazione ai bordi; (2) controllare la dimensione dell'output.
    if padding > 0:
        image = np.pad(
            image,
            ((padding, padding), (padding, padding)),  # pad su (alto,basso) e (sinistra,destra)
            mode='constant',
            constant_values=0
        )

    # H, W: altezza e larghezza dell'immagine (DOPO il padding, se applicato)
    H, W = image.shape

    # kH, kW: altezza e larghezza del kernel
    kH, kW = kernel.shape

    # Calcoliamo la dimensione spaziale dell'output in base a formula classica:
    # out = floor((in - kernel) / stride) + 1   (qui "in" è già H o W post-padding)
    outH = (H - kH) // stride + 1
    outW = (W - kW) // stride + 1

    # Inizializziamo la mappa di attivazione (feature map) di output a zeri
    out = np.zeros((outH, outW), dtype=np.float32)

    # Doppio ciclo per scorrere il kernel su ogni posizione valida dell'immagine
    for i in range(outH):           # indice verticale dell'output
        for j in range(outW):       # indice orizzontale dell'output
            # Estraggo il "patch" (la finestrella) dell'immagine sotto il kernel
            # La top-left del patch è (i*stride, j*stride) e la finestra è grande kH x kW
            patch = image[i*stride : i*stride + kH,
                          j*stride : j*stride + kW]

            # Prodotto elemento per elemento tra patch e kernel e somma → singolo numero in out[i, j]
            # Questo valore è alto quando il pattern "assomiglia" alle pesature del kernel (es. bordo verticale)
            out[i, j] = np.sum(patch * kernel)

    # Ritorniamo la feature map risultante
    return out

# Convoluzione SENZA padding e con stride=1
# L'output si restringe (perché il kernel "non entra" sui bordi): da 6x6 con kernel 3x3 ottieni 4x4
out_no_pad = conv2d_naive(img, kernel, stride=1, padding=0)

# Convoluzione CON padding=1 e stride=1
# Il padding "allarga" l'immagine a 8x8 prima della conv; con kernel 3x3 e stride=1 l'output torna 6x6.
# (Manteniamo la stessa dimensione spaziale dell'input "originale": comportamento detto "same")
out_pad_1 = conv2d_naive(img, kernel, stride=1, padding=1)

# Convoluzione CON padding=1 ma stride=2
# Lo stride=2 "salta" una cella a ogni passo → output più piccolo (downsampling): da 6x6 ottieni 3x3
out_stride_2 = conv2d_naive(img, kernel, stride=2, padding=1)

# Stampiamo le forme per verificare visivamente gli effetti di padding e stride
print("Forma input:", img.shape)
print("Senza padding, stride=1 →", out_no_pad.shape)     # atteso: (4, 4)
print("Padding=1, stride=1 →", out_pad_1.shape, "(mantiene dimensione)")  # atteso: (6, 6)
print("Padding=1, stride=2 →", out_stride_2.shape, "(uscita più piccola)") # atteso: (3, 3)

# Visualizziamo l'immagine di partenza (6x6) in scala di grigi
plt.figure()
plt.imshow(img, cmap='gray')     # 'gray' mostra 0=nero, 1=bianco
plt.title("Immagine giocattolo (6x6)")
plt.axis('off')                  # nascondiamo assi per pulizia
plt.show()

# Visualizziamo l'output della convoluzione con padding=1 e stride=1 (stessa dimensione dell'input)
# Qui ci aspettiamo valori alti in corrispondenza di bordi VERTICALI del quadrato bianco
plt.figure()
plt.imshow(out_pad_1, cmap='gray')
plt.title("Output convoluzione (pad=1, stride=1)")
plt.axis('off')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

# ==========================
# 1. Creiamo immagine e kernel
# ==========================
img = np.zeros((6, 6), dtype=np.float32)
img[2:4, 2:4] = 1.0  # piccolo quadrato bianco

kernel = np.array([[ 1,  0, -1],
                   [ 2,  0, -2],
                   [ 1,  0, -1]], dtype=np.float32)

stride = 1
padding = 1

# Applichiamo padding
padded_img = np.pad(img, ((padding, padding), (padding, padding)), mode='constant')
H, W = padded_img.shape
kH, kW = kernel.shape
outH = (H - kH)//stride + 1
outW = (W - kW)//stride + 1
out = np.zeros((outH, outW), dtype=np.float32)

# ==========================
# 2. Setup figura animata
# ==========================
plt.rcParams["animation.html"] = "jshtml"

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.set_title("Input + Kernel (con valori)")
ax2.set_title("Feature Map (valori aggiornati)")

im1 = ax1.imshow(padded_img, cmap='gray', vmin=0, vmax=1)
im2 = ax2.imshow(out, cmap='gray', vmin=-4, vmax=4)

# Rettangolo che rappresenta il kernel
rect = plt.Rectangle((0, 0), kW-1, kH-1, edgecolor='red', facecolor='none', lw=2)
ax1.add_patch(rect)

for ax in (ax1, ax2):
    ax.set_xticks([])
    ax.set_yticks([])

# Testo dinamico
text_kernel = fig.text(0.05, 0.02, "", fontsize=12, color='darkred')
text_value = fig.text(0.55, 0.02, "", fontsize=12, color='darkblue')

# Numeri della feature map
texts_out = [[ax2.text(j, i, "", ha="center", va="center", color="white", fontsize=10)
              for j in range(outW)] for i in range(outH)]

# Numeri del kernel che verranno visualizzati dentro il quadrato rosso
texts_kernel = [[ax1.text(0, 0, "", ha="center", va="center", color="yellow", fontsize=10)
                 for _ in range(kW)] for _ in range(kH)]

# ==========================
# 3. Funzione di aggiornamento
# ==========================
def update(frame):
    i = frame // outW
    j = frame % outW

    # estrai patch corrente
    patch = padded_img[i*stride : i*stride + kH,
                       j*stride : j*stride + kW]

    value = np.sum(patch * kernel)
    out[i, j] = value

    # aggiorna rettangolo
    rect.set_xy((j, i))
    im2.set_data(out)

    # aggiorna testi dinamici
    text_kernel.set_text(f"🧭 Calcolando output[{i},{j}]")
    text_value.set_text(f"📈 Σ(patch × kernel) = {value:.2f}")

    # aggiorna numeri sulla feature map
    for y in range(outH):
        for x in range(outW):
            texts_out[y][x].set_text(f"{out[y,x]:.1f}" if out[y,x] != 0 else "")

    # aggiorna numeri del kernel (centrati nella posizione corrente)
    for ki in range(kH):
        for kj in range(kW):
            x_pos = j + kj
            y_pos = i + ki
            texts_kernel[ki][kj].set_position((x_pos, y_pos))
            texts_kernel[ki][kj].set_text(f"{int(kernel[ki, kj])}")

    return [im1, im2, rect, text_kernel, text_value] + sum(texts_out, []) + sum(texts_kernel, [])

# ==========================
# 4. Animazione
# ==========================
ani = animation.FuncAnimation(
    fig, update, frames=outH*outW, interval=1000, blit=False, repeat=True
)

HTML(ani.to_jshtml())


# 🧭 Recap: cosa abbiamo fatto finora

Nel blocco precedente abbiamo:

1. **Creato un’immagine artificiale 6×6**, quasi tutta nera, con un piccolo quadrato bianco al centro.
→ Serve per osservare visivamente come i filtri rispondono ai bordi.

2. **Applicato una convoluzione 2D con un kernel(3×3)** che misura le variazioni di intensità orizzontali.

Il nostro kernel:

* colonne di sinistra positive (+1, +2, +1)

* colonne di destra negative (−1, −2, −1)
→ rileva bordi verticali, cioè transizioni nero ↔ bianco.

Visto l’effetto di padding e stride:

* Padding=0 → l’immagine “si restringe”

* Padding=1 → mantiene la stessa dimensione

* Stride=2 → dimezza l’output (downsampling)

Visualizzato l’output della convoluzione:

* Valori positivi (bianco) → bordo da scuro → chiaro

* Valori negativi (nero) → bordo da chiaro → scuro

* Valori vicini a 0 → regioni uniformi senza bordi

👉 In pratica, l’output rappresenta dove il filtro “vede” un bordo verticale.
Ogni valore della feature map è la risposta del filtro in quella posizione.

## 🧩 Passaggio successivo — Pooling

Ora applichiamo un pooling sull’output della convoluzione per:

1. ridurre la dimensione della feature map,

2. preservare le caratteristiche principali,

3. rendere il modello più robusto a piccoli spostamenti.

Esistono due tipi principali di pooling:

* **Max Pooling:** prende il valore massimo di ogni blocco (conserva le attivazioni più forti)

* **Average Pooling:** prende la media di ogni blocco (effetto di “lisciatura”)

Visivamente ⬇️

![Alt text](https://miro.medium.com/v2/resize:fit:640/format:webp/1*sIf_zGTXgvTEDkpeP8jvxg.jpeg)




In [ ]:
# ============================================================
# 📘 DIMOSTRAZIONE: Operatori di Manipolazione Tensors in PyTorch
# ============================================================
import torch

# Creiamo un tensore di esempio
x = torch.tensor([[[1], [2], [3]]])  # shape: (1, 3, 1)
print("Tensor iniziale:")
print(x)
print("Shape iniziale:", x.shape)


x_squeezed = torch.squeeze(x)
print("\n🔹 Dopo squeeze:")
print(x_squeezed)
print("Shape:", x_squeezed.shape)


x_squeezed_dim0 = torch.squeeze(x, dim=0)
print("\n🔹 squeeze(dim=0):", x_squeezed_dim0.shape)


x_unsqueezed = torch.unsqueeze(x_squeezed, dim=0)
print("\n🔹 Dopo unsqueeze(dim=0):")
print(x_unsqueezed)
print("Shape:", x_unsqueezed.shape)


x2 = torch.arange(1, 9)  # [1,2,...,8]
print("\nTensor base:", x2)
print("Shape:", x2.shape)

x2_view = x2.view(2, 4)
x2_reshape = x2.reshape(4, 2)
print("\n🔹 view(2,4): shape", x2_view.shape)
print("🔹 reshape(4,2): shape", x2_reshape.shape)


x3 = torch.randn(2, 3, 4)
print("\nShape originale:", x3.shape)

x3_T = x3.transpose(0, 1)
print("🔹 transpose(0,1):", x3_T.shape)

x3_P = x3.permute(2, 0, 1)
print("🔹 permute(2,0,1):", x3_P.shape)


x4 = torch.tensor([[1], [2], [3]])  # shape: (3,1)
print("\nTensor base:\n", x4)

x_expand = x4.expand(3, 4)
x_repeat = x4.repeat(1, 4)

print("\n🔹 expand(3,4):\n", x_expand)
print("Shape:", x_expand.shape)

print("\n🔹 repeat(1,4):\n", x_repeat)
print("Shape:", x_repeat.shape)


print("\n📊 Riepilogo finale:")
print("x shape:", x.shape)
print("squeeze →", x_squeezed.shape)
print("unsqueeze →", x_unsqueezed.shape)
print("view →", x2_view.shape)
print("transpose →", x3_T.shape)
print("permute →", x3_P.shape)
print("expand →", x_expand.shape)
print("repeat →", x_repeat.shape)



In [ ]:
import torch
import torch.nn as nn

# Convertiamo l'output della convoluzione (out_pad_1, un array NumPy 6x6) in un TENSORE PyTorch.
# Le reti convoluzionali in PyTorch si aspettano input con forma:
# (N, C, H, W)
# dove:
#   N = numero di esempi (batch size)
#   C = numero di canali (es. 1 per immagini in scala di grigi, 3 per RGB)
#   H = altezza (height)
#   W = larghezza (width)

# out_pad_1 ha shape (6, 6) → solo H e W.
# Per renderlo compatibile, aggiungiamo:
# - una dimensione per il batch (N=1)
# - una per il canale (C=1)
# Risultato: (1, 1, 6, 6)
feature_map = torch.tensor(out_pad_1, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
print("Shape feature map:", feature_map.shape)  # (1, 1, 6, 6)

# Definiamo i due tipi di pooling:
# MaxPool2d: seleziona il valore massimo in ogni finestra (kernel 2x2)
# AvgPool2d: calcola la media dei valori nella stessa finestra
# Lo stride=2 significa che ogni finestra "salta" di 2 pixel → dimezza H e W (6x6 → 3x3)
maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
avgpool = nn.AvgPool2d(kernel_size=2, stride=2)

# Applichiamo il pooling alla feature map
max_pooled = maxpool(feature_map)  # mantiene la forma (1,1,3,3)
avg_pooled = avgpool(feature_map)

# Stampiamo le forme per verificare la riduzione spaziale
print("Max Pooling output shape:", max_pooled.shape)
print("Average Pooling output shape:", avg_pooled.shape)

# Visualizziamo i risultati a confronto:
# - Feature map originale (output della convoluzione)
# - Max pooling (3x3): mantiene solo i valori massimi locali
# - Average pooling (3x3): calcola la media locale, smussando i dettagli
fig, axes = plt.subplots(1, 3, figsize=(10, 3))

axes[0].imshow(out_pad_1, cmap='gray')
axes[0].set_title("Feature map originale (6×6)")

axes[1].imshow(max_pooled.squeeze(), cmap='gray')
axes[1].set_title("Max pooling (3×3)")

axes[2].imshow(avg_pooled.squeeze(), cmap='gray')
axes[2].set_title("Average pooling (3×3)")

# Rimuoviamo gli assi per una visualizzazione più pulita
for ax in axes:
    ax.axis('off')

plt.show()


# 📊 Cosa osserviamo

* Dopo il **Max Pooling**, l’immagine è più piccola (6×6 → 3×3) ma i bordi principali sono ancora ben evidenti.

* Dopo **l’Average Pooling**, le variazioni si “ammorbidiscono”: le intensità estreme vengono mediate.

✅ Conclusione

Il pooling serve per:

* ridurre la quantità di dati da elaborare nei layer successivi,

* mantenere solo le informazioni più significative,

* garantire invarianza a traslazioni locali (es. un bordo spostato di 1 pixel non cambia molto l’attivazione del blocco).

In una CNN reale, dopo ogni convoluzione si alternano tipicamente:

**Conv → ReLU → Pooling**

per costruire progressivamente rappresentazioni più compatte ma più ricche di significato.


## La tua prima CNN (MNIST)

Costruiremo una piccola CNN per **MNIST** (cifre 28×28 in scala di grigi).  
**Pipeline:**
1. **Transforms**: conversione a tensore e **normalizzazione** (media=0.1307, std=0.3081).  
2. **Dataset & DataLoader** per train/val/test.  
3. **Modello**: due blocchi conv → flatten → fully connected.  
4. **Training loop** con `CrossEntropyLoss` + `Adam`.  
5. **Accuratezza** e barra `tqdm` per il progresso.  
6. **Grafici** di loss e accuracy.


In [ ]:
# 3.1 Trasformazioni (normalizzazione consigliata per MNIST)
# Compose concatena più trasformazioni da applicare in sequenza a ogni immagine.
# ToTensor: converte un'immagine PIL/array [0..255] in un tensore float [0..1] con shape (C,H,W).
# Normalize: per ogni canale applica (x - mean) / std. Per MNIST (grayscale) c'è un solo canale.
#            mean=0.1307 e std=0.3081 sono statistiche standard del dataset MNIST.
transform_mnist = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # media e std note per MNIST
])

# 3.2 Dataset (con fallback)
# Directory locale dove scaricare o cercare i file del dataset.
data_root = "./data"

# Proviamo a caricare/scaricare MNIST:
# - train=True: split di addestramento (60k immagini)
# - train=False: split di test (10k immagini)
# - transform=transform_mnist: applica le trasformazioni definite sopra
# - download=True: scarica se non è già presente nella cartella data_root
try:
    train_full = datasets.MNIST(root=data_root, train=True,  transform=transform_mnist, download=True)
    test_ds    = datasets.MNIST(root=data_root, train=False, transform=transform_mnist, download=True)
except Exception as e:
    # In caso di assenza di rete o problemi di I/O, si entra qui.
    # ATTENZIONE: con questo solo print, train_full e test_ds rimangono NON definiti.
    # Più sotto trovi una variante robusta che crea un vero fallback.
    print("⚠️ Impossibile scaricare MNIST. Errore:", str(e))

# 3.3 Split train/val
# Percentuale del training set da usare come validazione (10%).
val_ratio = 0.1
# Numero di esempi destinati alla validazione.
val_size = int(len(train_full) * val_ratio)
# Numero di esempi destinati al training vero e proprio.
train_size = len(train_full) - val_size
# random_split crea due sottoinsiemi casuali del dataset originale.
# (La casualità è controllata dal seed impostato in precedenza per riproducibilità.)
train_ds, val_ds = random_split(train_full, [train_size, val_size])

# 3.4 DataLoader
# Dimensione dei batch (128 è un buon compromesso per MNIST; puoi aumentarla con GPU più capienti).
batch_size = 128
# DataLoader per il training: shuffle=True rimescola i campioni a ogni epoca (buona pratica).
# num_workers=2 usa due processi per il prefetch dei dati (aumenta throughput, valuta in base alla macchina).
# pin_memory=True su CUDA fissa le pagine in RAM per copie host→device più rapide.
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,
                          num_workers=2, pin_memory=True if device.type=='cuda' else False)
# DataLoader per la validazione: niente shuffle (ordine stabile).
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False,
                          num_workers=2, pin_memory=True if device.type=='cuda' else False)
# DataLoader per il test: niente shuffle (ordine stabile).
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False,
                          num_workers=2, pin_memory=True if device.type=='cuda' else False)

# In un notebook, l'ultima espressione non assegnata viene mostrata come output della cella.
# Qui riportiamo le cardinalità dei tre split e una stringa "MNIST" come etichetta di comodo.
len(train_ds), len(val_ds), len(test_ds), "MNIST"


In [ ]:
# 3.5 Visualizziamo alcune immagini con le etichette
def show_batch(dl, n=5):
    # Stampa la lunghezza del DataLoader, ovvero il numero totale di batch.
    # Esempio: se il dataset ha 54.000 immagini e batch_size=128 → ci saranno circa 422 batch.
    print(len(dl))

    # next(iter(dl)) estrae il PRIMO batch dal DataLoader.
    # xb = immagini (batch di tensori)
    # yb = etichette (batch di target numerici)
    xb, yb = next(iter(dl))

    # Stampa le shape dei tensori per capire come sono strutturati.
    print(xb.shape)  # (batch_size, canali, altezza, larghezza)
    print(yb.shape)  # (batch_size,)

    # Seleziona solo i primi n campioni del batch (es. 5 immagini)
    xb, yb = xb[:n], yb[:n]

    # Sposta i dati su CPU e li converte in array NumPy per poterli plottare con Matplotlib.
    xb = xb.cpu().numpy()

    # Crea una nuova figura.
    plt.figure()

    # cols = numero di colonne nel layout dei subplot (una riga con n immagini)
    cols = n
    for i in range(n):
        # Crea una subplot 1×n, posizione i+1
        plt.subplot(1, cols, i+1)

        # xb[i][0]: seleziona la i-esima immagine e il primo canale (0, perché è in bianco e nero)
        plt.imshow(xb[i][0], cmap='gray')

        # Mostra il numero della classe come titolo
        plt.title(f"Label: {int(yb[i])}")

        # Nasconde gli assi per una visualizzazione più pulita
        plt.axis('off')

    # Mostra tutte le immagini insieme
    plt.show()

# Esegui la funzione per visualizzare 5 immagini dal training set
show_batch(train_loader, n=5)


In [ ]:
# 3.6 Definiamo la CNN
class SimpleCNN(nn.Module):  # Definiamo una rete neurale come sottoclasse di nn.Module (base class PyTorch per i modelli)
    def __init__(self, p_dropout=0.0, use_batchnorm=False):
        super().__init__()  # Inizializza la parte nn.Module (registrazione dei sottolayer, ecc.)

        # c1 e c2 = numero di canali (feature maps) prodotti dai due blocchi convoluzionali.
        # Valori piccoli (5 e 10) per chiarezza didattica: più canali = più capacità, più costi.
        c1, c2 = 5, 10

        # Costruiamo progressivamente la "pila" di layer convoluzionali/attivazioni/pooling.
        layers = []

        # --- BLOCCO CONV 1 ---
        # nn.Conv2d(in_channels=1, out_channels=c1, kernel_size=3, padding=1)
        # - in_channels=1: immagini MNIST sono 1-canale (grayscale).
        # - out_channels=c1: n° di filtri appresi (uscita: c1 mappe di attivazione).
        # - kernel_size=3: filtri 3x3 (standard).
        # - padding=1: "same-ish": mantiene la dimensione spaziale (28 -> 28) con kernel=3.
        layers += [nn.Conv2d(1, c1, kernel_size=3, padding=1),
                   nn.ReLU(inplace=True)]  # ReLU rende non lineare. inplace=True: risparmia memoria (sovrascrive input).

        # Facoltativo: BatchNorm2d stabilizza/accelera il training normalizzando le attivazioni per canale.
        # (Si applica dopo conv e prima/ dopo ReLU; qui dopo ReLU per semplicità didattica)
        if use_batchnorm:
            layers += [nn.BatchNorm2d(c1)]

        # MaxPool2d(2,2): dimezza le dimensioni spaziali (H, W) prendendo il massimo in finestre 2x2.
        # 28x28 -> 14x14
        layers += [nn.MaxPool2d(2, 2)]  # 28->14

        # --- BLOCCO CONV 2 ---
        # Seconda convoluzione:
        # - in_channels=c1 (quanti canali escono dal blocco 1)
        # - out_channels=c2 (nuove mappe di attivazione)
        # - padding=1 mantiene 14x14 in ingresso -> 14x14 in uscita prima del pooling
        layers += [nn.Conv2d(c1, c2, kernel_size=3, padding=1),
                   nn.ReLU(inplace=True)]

        if use_batchnorm:
            layers += [nn.BatchNorm2d(c2)]

        # Secondo MaxPool: 14x14 -> 7x7
        layers += [nn.MaxPool2d(2, 2)]  # 14->7

        # Raggruppiamo i layer feature-extractor in un Sequential per rendere il forward compatto/leggibile.
        self.features = nn.Sequential(*layers)

        # Dropout sul vettore flatten prima del classificatore:
        # - p_dropout=0.0: disattivato; >0.0 attivato (es. 0.3)
        # - Dropout aiuta a ridurre overfitting azzerando casualmente alcune attivazioni a train-time.
        # - nn.Identity() è un "layer no-op" (fa pass-through) quando non vogliamo dropout.
        self.dropout = nn.Dropout(p_dropout) if p_dropout > 0 else nn.Identity()

        # Layer fully-connected (classificatore):
        # - Input atteso: vettore di dimensione 7*7*c2 (dopo i due MaxPool).
        #   * Partiamo da 28x28 -> Conv (28x28) -> Pool (14x14) -> Conv (14x14) -> Pool (7x7)
        #   * Canali finali = c2
        # - Output: 10 classi (cifre 0..9) per MNIST.
        self.classifier = nn.Linear(7 * 7 * c2, 10)

    def forward(self, x):
        # x forma attesa: (N, 1, 28, 28)  N=batch size, 1=grayscale
        x = self.features(x)          # Applica i blocchi Conv/ReLU/(BN)/Pool -> output: (N, c2, 7, 7)
        x = torch.flatten(x, 1)       # Flatten a partire dalla dim=1 (mantiene N): (N, 7*7*c2)
        x = self.dropout(x)           # Eventuale dropout (se p_dropout>0), altrimenti pass-through
        x = self.classifier(x)        # Linear -> logits di dimensione 10 (uno per classe)
        return x                      # Ritorniamo logits (NON softmax). CrossEntropyLoss applica softmax internamente.

def count_params(model):
    # Conta il numero TOTALE di parametri allenabili (requires_grad=True).
    # p.numel() = n° elementi del tensore (es. un peso per ogni connessione/filtro).
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Istanzia il modello:
# - p_dropout=0.0: nessun dropout (metti 0.3 per iniziare a regolarizzare)
# - use_batchnorm=False: BN disattivata (metti True per attivarla)
# - .to(device): sposta i pesi su GPU se disponibile, altrimenti CPU
model = SimpleCNN(p_dropout=0.0, use_batchnorm=False).to(device)

print(model)  # Stampa l’architettura leggibile (layer per layer con shape implicite)

# Conta e stampa quanti parametri verranno effettivamente aggiornati durante il training
print("Parametri allenabili:", count_params(model))


In [ ]:
# Calcola l'accuratezza a partire dai logits (uscita del modello) e dalle label vere y
def accuracy(logits, y):
    # logits: tensore di forma (N, C) — N=batch size, C=numero classi (es. 10)
    # argmax(dim=1): prende l'indice della classe con logit più alto per ciascun esempio → shape (N,)
    preds = logits.argmax(dim=1)
    # (preds == y): tensore booleano (N,) True/False per ogni esempio
    # .float(): converte True→1.0, False→0.0
    # .mean(): media sui N esempi → accuratezza in [0,1]
    # .item(): estrae lo scalare Python dal tensore (utile per log / accumulo)
    return (preds == y).float().mean().item()


# Esegue UNA epoca di training (forward + backward + update pesi) e restituisce loss/accuracy medie sull'epoca
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()  # modalità training: attiva dropout, BatchNorm usa statistiche di batch, ecc.
    running_loss, running_acc = 0.0, 0.0  # accumulatori per somma dei contributi (poi faremo la media pesata)
    # tqdm: barra di avanzamento sul DataLoader; desc è il testo mostrato; leave=False non lascia la barra dopo il loop
    for xb, yb in tqdm(loader, desc="Train", leave=False):
        # Sposta batch (immagini) e label sul device corretto (CPU/GPU)
        xb, yb = xb.to(device), yb.to(device)

        optimizer.zero_grad()   # azzera i gradienti accumulati dal passo precedente (PyTorch li accumula di default)

        logits = model(xb)      # forward pass: otteniamo i logits (shape (N, C))
        loss = criterion(logits, yb)  # calcolo della loss (es. CrossEntropyLoss sui logits e le label)

        loss.backward()         # backpropagation: calcola i gradienti dL/dθ
        optimizer.step()        # aggiornamento dei pesi secondo la regola dell’ottimizzatore (Adam/SGD, ecc.)

        # Accumulo della SOMMA della loss pesata per il numero di esempi nel batch
        # .item() estrae lo scalare Python; xb.size(0) = N del batch (ultimo batch può essere più piccolo)
        running_loss += loss.item() * xb.size(0)

        # Accumulo della SOMMA delle accuratezze per esempio (accuracy(batch)*N)
        # In questo modo la media finale è correttamente pesata per la dimensione dei batch
        running_acc  += accuracy(logits, yb) * xb.size(0)

    n = len(loader.dataset)      # numero totale di esempi visti in questa epoca (tutti i batch)
    # Ritorniamo le MEDIE su tutti gli esempi (loss e accuracy micro-averaged)
    return running_loss / n, running_acc / n


# Valutazione (validazione/test): NESSUN gradiente, NESSUN update pesi
@torch.no_grad()  # disattiva il tracciamento dei gradienti all'interno della funzione (meno memoria, più veloce)
def evaluate(model, loader, criterion, phase="Val"):
    model.eval()  # modalità eval: disattiva dropout; BatchNorm usa le running stats (non quelle del batch)
    running_loss, running_acc = 0.0, 0.0
    # Loop sui batch del loader (val o test); tqdm mostra "Val" o "Test" a seconda del parametro
    for xb, yb in tqdm(loader, desc=phase, leave=False):
        xb, yb = xb.to(device), yb.to(device)   # sposta dati su device
        logits = model(xb)                      # forward pass (solo inferenza)
        loss = criterion(logits, yb)            # calcolo loss di validazione/test

        # Accumuli come nel training, ma senza backward/step
        running_loss += loss.item() * xb.size(0)
        running_acc  += accuracy(logits, yb) * xb.size(0)

    n = len(loader.dataset)  # numero totale di esempi del set (val/test)
    # Medie su tutto il dataset (corrette anche se l'ultimo batch è più piccolo)
    return running_loss / n, running_acc / n


In [ ]:

# 3.8 Addestriamo!
epochs = 3
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4, weight_decay=0.0)  # weight_decay è la L2
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)  # semplice scheduler LR

history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

for epoch in range(1, epochs+1):
    print(f"\nEpoca {epoch}/{epochs}")
    tl, ta = train_one_epoch(model, train_loader, optimizer, criterion)
    vl, va = evaluate(model, val_loader, criterion, phase="Val")
    scheduler.step()

    history["train_loss"].append(tl); history["train_acc"].append(ta)
    history["val_loss"].append(vl);   history["val_acc"].append(va)

    print(f"loss_train={tl:.4f}  acc_train={ta:.4f} | loss_val={vl:.4f}  acc_val={va:.4f}")


In [ ]:

# 3.9 Grafici delle curve di apprendimento (matplotlib; un grafico per figura; nessuno stile colore specifico)
plt.figure()
plt.plot(history["train_loss"], label="loss_train")
plt.plot(history["val_loss"],   label="loss_val")
plt.legend(); plt.xlabel("epoca"); plt.ylabel("loss"); plt.title("Andamento della loss"); plt.show()

plt.figure()
plt.plot(history["train_acc"], label="acc_train")
plt.plot(history["val_acc"],   label="acc_val")
plt.legend(); plt.xlabel("epoca"); plt.ylabel("accuratezza"); plt.title("Andamento dell'accuratezza"); plt.show()


In [ ]:

# 3.10 Valutazione sul test set + alcune predizioni
@torch.no_grad()
def get_predictions(model, loader, max_batches=1):
    model.eval()
    images, labels, preds = [], [], []
    batches_done = 0
    for xb, yb in loader:
        xb = xb.to(device)
        logits = model(xb)
        pb = logits.argmax(dim=1).cpu()
        images.append(xb.cpu())
        labels.append(yb)
        preds.append(pb)
        batches_done += 1
        if batches_done >= max_batches:
            break
    return torch.cat(images, dim=0), torch.cat(labels, dim=0), torch.cat(preds, dim=0)

test_loss, test_acc = evaluate(model, test_loader, nn.CrossEntropyLoss(), phase="Test")
print(f"Test: loss={test_loss:.4f}, acc={test_acc:.4f}")

imgs, labs, prds = get_predictions(model, test_loader, max_batches=1)
n = min(8, imgs.size(0))
plt.figure()
for i in range(n):
    plt.subplot(1, n, i+1)
    plt.imshow(imgs[i,0], cmap='gray')
    plt.title(f"T:{int(labs[i])}/P:{int(prds[i])}")
    plt.axis('off')
plt.show()



## 4. Regolarizzazione e miglioramenti

**Perché servono?** Per ridurre **overfitting** e rendere l'addestramento più stabile.

- **Dropout:** azzera casualmente alcune attivazioni durante il training (es. `p=0.3`).  
- **Weight decay (L2):** penalizza pesi grandi tramite l'ottimizzatore (es. `weight_decay=1e-4`).  
- **Batch Normalization:** normalizza le attivazioni per velocizzare/stabilizzare il training.  
- **Scheduler LR:** regola il learning rate durante l'addestramento (es. `StepLR`, `OneCycleLR`).

Sotto attiviamo **dropout + batch norm** e facciamo un breve training dimostrativo.


In [ ]:

model_bn = SimpleCNN(p_dropout=0.3, use_batchnorm=True).to(device)
optimizer = optim.Adam(model_bn.parameters(), lr=3e-4, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

epochs = 2
for epoch in range(1, epochs+1):
    print(f"\n[BN+Dropout] Epoca {epoch}/{epochs}")
    tl, ta = train_one_epoch(model_bn, train_loader, optimizer, criterion)
    vl, va = evaluate(model_bn, val_loader, criterion, phase="Val")
    print(f"loss_train={tl:.4f}  acc_train={ta:.4f} | loss_val={vl:.4f}  acc_val={va:.4f}")




## 4bis. Confusion Matrix ed Early Stopping

In questa sezione impariamo due strumenti fondamentali per migliorare l’analisi e la robustezza del modello:

1. **Confusion Matrix (Matrice di confusione)**  
   Serve per capire **quali classi il modello confonde** tra loro.  
   Ogni cella mostra quante volte una classe reale è stata predetta come un’altra classe.

   - La diagonale rappresenta le **predizioni corrette**.  
   - Le celle fuori diagonale mostrano gli **errori di classificazione**.

2. **Early Stopping (Interruzione anticipata)**  
   Serve a fermare l’addestramento **prima** che il modello inizi a sovradattarsi (*overfitting*).  
   Se la **loss di validazione** non migliora dopo un certo numero di epoche (pazienza), si interrompe il training.


In [ ]:
# Importiamo due utility da scikit-learn:
# - confusion_matrix: calcola la matrice di confusione a partire da etichette vere e predette
# - ConfusionMatrixDisplay: oggetto comodo per visualizzare la matrice con Matplotlib
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Disattiviamo il tracciamento del gradiente dentro questa funzione (niente backprop durante la valutazione):
@torch.no_grad()
def compute_confusion_matrix(model, loader):
    model.eval()                       # Mettiamo il modello in modalità 'eval' (disattiva dropout, BN in mode eval, ecc.)
    all_preds, all_labels = [], []     # Liste dove accumuliamo predizioni e label di TUTTI i batch

    # Iteriamo su tutto il DataLoader (tipicamente test_loader)
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)   # Spostiamo batch e label sul device corretto (CPU/GPU)
        preds = model(xb).argmax(1)             # Forward pass → logits; argmax(1) prende la classe con logit più alto per ogni immagine
        all_preds.append(preds.cpu())           # Portiamo su CPU e accodiamo le predizioni del batch
        all_labels.append(yb.cpu())             # Portiamo su CPU e accodiamo le label vere del batch

    # Concateniamo tutte le predizioni/etichette in un unico tensore (dimensione totale = n. esempi del loader)
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    # Calcoliamo la matrice di confusione: righe = etichetta vera, colonne = etichetta predetta
    cm = confusion_matrix(all_labels, all_preds)
    return cm

# --- Calcolo e visualizzazione della matrice di confusione sul TEST set ---

cm = compute_confusion_matrix(model, test_loader)       # Otteniamo la matrice (10x10 per MNIST)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,      # Prepariamo l'oggetto di visualizzazione
                              display_labels=list(range(10)))  # Etichette degli assi: 0..9
disp.plot(cmap="Blues", values_format="d")              # Disegniamo la heatmap in blu; 'd' = interi nelle celle
plt.title("Matrice di confusione - MNIST")              # Titolo del grafico
plt.show()                                              # Mostriamo la figura


In [ ]:
# Early Stopping semplice: si interrompe se la loss di validazione non migliora per 'patience' epoche consecutive

def train_with_early_stopping(model, train_loader, val_loader, criterion, optimizer, scheduler=None, epochs=20, patience=3, min_delta=0.05 ):#1e-3):
    # Inizializziamo la "migliore" loss di validazione con infinito (qualsiasi valore reale sarà più piccolo)
    best_val_loss = float('inf')
    # Contatore di epoche consecutive senza miglioramento
    counter = 0
    # Dizionario per tenere traccia delle metriche nel tempo (per plotting/analisi)
    history = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}
    best_model_wts = copy.deepcopy(model.state_dict())  # Pesi iniziali (in caso non migliori mai)

    # Loop sulle epoche
    for epoch in range(1, epochs+1):
        # Messaggio di stato per chiarezza in output
        print(f"\nEpoca {epoch}/{epochs}")

        # Eseguiamo un'epoca di training (funzione definita prima: imposta model.train(), fa i batch, backprop, ecc.)
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
        # Eseguiamo la valutazione sul validation set (funzione definita prima: model.eval(), no grad, calcolo loss/acc)
        val_loss, val_acc = evaluate(model, val_loader, criterion, phase='Val')

        # Se abbiamo uno scheduler di learning rate "per epoca", lo facciamo avanzare qui
        if scheduler: scheduler.step()

        # Salviamo le metriche dell'epoca nel nostro storico
        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["train_acc"].append(train_acc)
        history["val_acc"].append(val_acc)

        # Log leggibile della qualità raggiunta in questa epoca
        print(f"loss_train={train_loss:.4f} | loss_val={val_loss:.4f}")

        # -------------------------
        #      EARLY STOPPING
        # -------------------------
        # --- Early Stopping Logic ---
        improvement = best_val_loss - val_loss
        # Se la loss di validazione è migliorata, azzeriamo il contatore e "salviamo" i pesi migliori
        if improvement > min_delta:
            best_val_loss = val_loss
            counter = 0
            best_model_wts = copy.deepcopy(model.state_dict())
            print(f"✅ Miglioramento rilevato: nuova best_val_loss = {best_val_loss:.4f}")
        else:
            # Nessun miglioramento sufficiente
            counter += 1
            print(f"⚠️ Nessun miglioramento significativo ({counter}/{patience})")
            if counter >= patience:
                print(f"🛑 Early Stopping attivato dopo {epoch} epoche! Miglior val_loss = {best_val_loss:.4f}")
                model.load_state_dict(best_model_wts)
                break

    # Ritorniamo il modello (con i migliori pesi) e lo storico delle metriche
    return model, history

# Esempio pratico (allenamento breve con early stopping)
# Istanzio una SimpleCNN con un po' di regolarizzazione (Dropout) e BatchNorm attiva
model_es = SimpleCNN(p_dropout=0.2, use_batchnorm=True).to(device)
# Ottimizzatore Adam con LR moderato
optimizer = optim.Adam(model_es.parameters(), lr=3e-4)
# Loss per classificazione multi-classe: accetta logits e applica softmax internamente
criterion = nn.CrossEntropyLoss()
# Alleno per max 10 epoche, ma fermo prima se per 2 epoche di fila la val_loss non migliora
model_es, hist_es = train_with_early_stopping(model_es, train_loader, val_loader, criterion, optimizer, epochs=10, patience=2)



### Visualizzazione delle curve di Early Stopping

Nei grafici seguenti vediamo come si comportano **loss** e **accuratezza** nel tempo:  
- le curve blu rappresentano l’andamento del **training**;  
- le curve arancioni rappresentano l’**insieme di validazione**;  
- il **punto rosso** indica il momento in cui l’early stopping ha interrotto l’addestramento.

Osservando i grafici puoi capire quando il modello ha smesso di migliorare sulla validazione, pur continuando a migliorare (overfitting) sul training.


In [ ]:

# Troviamo l'epoca di stop (ultima epoca effettiva eseguita)
stopped_epoch = len(hist_es["val_loss"]) - 1

plt.figure()
plt.plot(hist_es["train_loss"], label="loss_train")
plt.plot(hist_es["val_loss"], label="loss_val")
plt.scatter(stopped_epoch, hist_es["val_loss"][stopped_epoch], color="red", label="stop")
plt.legend(); plt.xlabel("epoca"); plt.ylabel("loss")
plt.title("Early Stopping - Andamento della loss")
plt.show()

plt.figure()
plt.plot(hist_es["train_acc"], label="acc_train")
plt.plot(hist_es["val_acc"], label="acc_val")
plt.scatter(stopped_epoch, hist_es["val_acc"][stopped_epoch], color="red", label="stop")
plt.legend(); plt.xlabel("epoca"); plt.ylabel("accuratezza")
plt.title("Early Stopping - Andamento dell'accuratezza")
plt.show()



## 5. Salvataggio e caricamento del modello

Usa `state_dict` per salvare solo i pesi:


In [ ]:

save_path = "simple_cnn_mnist.pth"
torch.save(model.state_dict(), save_path)
print(f"Salvato in {save_path}")

# Per caricare:
loaded = SimpleCNN()
loaded.load_state_dict(torch.load(save_path, map_location="cpu"))
loaded.eval()
print("Pesi caricati in un nuovo modello.")



## 6. (Opzionale) Assaggio di transfer learning (CIFAR-10)

**Idea:** partire da un modello pre-addestrato su un grande dataset (es. ImageNet) e adattarlo al tuo compito.

Useremo **ResNet18** da `torchvision.models`. Se non è possibile scaricare il dataset, ricorriamo a `FakeData` (3×32×32, ridimensionate a 224×224).  
È solo un assaggio: per risultati solidi servono più epoche e un setup più curato.


In [ ]:
# Importiamo la ResNet18 e i pesi pre-addestrati disponibili in torchvision
from torchvision.models import resnet18, ResNet18_Weights

# ----------------------------
# 1️⃣ Definizione delle trasformazioni
# ----------------------------
transform_cifar = transforms.Compose([
    # CIFAR-10 ha immagini 32x32, ma ResNet18 si aspetta input 224x224 (ImageNet)
    transforms.Resize((224, 224)),

    # Converte l'immagine PIL in tensore PyTorch con valori [0,1]
    transforms.ToTensor(),

    # Normalizzazione con le stesse statistiche (mean/std) di ImageNet
    # fondamentale per far funzionare bene i pesi pre-addestrati
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225))
])

# ----------------------------
# 2️⃣ Caricamento dataset CIFAR-10 (con fallback)
# ----------------------------
try:
    # Split di training
    cifar_train = datasets.CIFAR10(root="./data", train=True,
                                   transform=transform_cifar, download=True)
    # Split di test
    cifar_test  = datasets.CIFAR10(root="./data", train=False,
                                   transform=transform_cifar, download=True)
except Exception as e:
    # In caso di problemi di rete o permessi, fallback su FakeData per non bloccare il notebook
    print("⚠️ Impossibile scaricare CIFAR-10. Fallback su FakeData. Errore:", str(e))
    cifar_train = datasets.FakeData(size=50000, image_size=(3,224,224),
                                    num_classes=10, transform=transform_cifar)
    cifar_test  = datasets.FakeData(size=10000, image_size=(3,224,224),
                                    num_classes=10, transform=transform_cifar)

# ----------------------------
# 3️⃣ Creazione DataLoader
# ----------------------------
# batch_size=64 → 64 immagini per batch
# shuffle=True nel training → mescola i dati a ogni epoca per migliorare la generalizzazione
# pin_memory=True (solo su GPU): migliora le prestazioni delle copie CPU→GPU
cifar_train_loader = DataLoader(cifar_train, batch_size=64, shuffle=True,
                                num_workers=2, pin_memory=True if device.type=='cuda' else False)

# Per il test non serve shuffle (l'ordine non influisce)
cifar_test_loader  = DataLoader(cifar_test,  batch_size=64, shuffle=False,
                                num_workers=2, pin_memory=True if device.type=='cuda' else False)

# ----------------------------
# 4️⃣ Caricamento modello pre-addestrato
# ----------------------------
# Otteniamo i pesi predefiniti di ResNet18 (pretrained su ImageNet)
weights = ResNet18_Weights.DEFAULT

# Carichiamo la rete con quei pesi
backbone = resnet18(weights=weights)

# ----------------------------
# 5️⃣ Congelamento dei layer di feature extraction
# ----------------------------
# In questo modo NON aggiorniamo i pesi delle convoluzioni già apprese su ImageNet
# (evitiamo di distruggere conoscenze generali come rilevatori di bordi, texture, forme)
for param in backbone.parameters():
    param.requires_grad = False

# ----------------------------
# 6️⃣ Sostituzione della testa di classificazione
# ----------------------------
# ResNet18 pre-addestrata ha un layer finale (fc) con 1000 output (per ImageNet)
# Lo sostituiamo con un layer fully connected per 10 classi (CIFAR-10)
num_feats = backbone.fc.in_features   # Numero di feature in input alla fc (512 per ResNet18)
backbone.fc = nn.Linear(num_feats, 10)

# Spostiamo tutto il modello sul device (CPU o GPU)
backbone = backbone.to(device)

# ----------------------------
# 7️⃣ Ottimizzatore e loss function
# ----------------------------
# Addestriamo SOLO la nuova testa (gli altri layer restano congelati)
optimizer = optim.Adam(backbone.fc.parameters(), lr=1e-3)
# Loss per classificazione multi-classe
criterion = nn.CrossEntropyLoss()

# ----------------------------
# 8️⃣ Mini addestramento dimostrativo
# ----------------------------
# Solo 1 epoca per mostrare come funziona il transfer learning
epoch = 1
print("\n[Transfer Learning] Demo veloce")

# Eseguiamo un’epoca di training sul train_loader
tl, ta = train_one_epoch(backbone, cifar_train_loader, optimizer, criterion)
# E valutiamo sul test_loader
vl, va = evaluate(backbone, cifar_test_loader, criterion, phase="Test")

# Stampiamo le metriche di performance
print(f"loss_train={tl:.4f}  acc_train={ta:.4f} | loss_test={vl:.4f}  acc_test={va:.4f}")


In [ ]:
# Classi di CIFAR-10
cifar_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                 'dog', 'frog', 'horse', 'ship', 'truck']

def predict_and_show(model, loader, n=5):
    """
    Mostra n immagini del loader con la predizione del modello.
    """
    model.eval()  # disattiva dropout e batchnorm in modalità training
    xb, yb = next(iter(loader))          # prende il primo batch
    xb, yb = xb.to(device), yb.to(device)

    with torch.no_grad():                # no gradient → più veloce, meno memoria
        preds = model(xb).argmax(1)      # indice della classe con logit più alto

    xb = xb.cpu().permute(0, 2, 3, 1)    # da (N,C,H,W) → (N,H,W,C) per Matplotlib
    xb = xb * torch.tensor((0.229, 0.224, 0.225)) + torch.tensor((0.485, 0.456, 0.406))
    xb = torch.clamp(xb, 0, 1)           # de-normalizzazione per visualizzare correttamente

    plt.figure(figsize=(12, 4))
    for i in range(n):
        plt.subplot(1, n, i+1)
        plt.imshow(xb[i])
        plt.title(f"True: {cifar_classes[yb[i]]}\nPred: {cifar_classes[preds[i]]}",
                  fontsize=10)
        plt.axis('off')
    plt.show()

# 🔍 Eseguiamo la funzione di predizione
predict_and_show(backbone, cifar_test_loader, n=5)



## 7. Forme (shape) e trucchi di debug

**Dimensione di uscita di `Conv2d`** (per una sola dimensione):  
\[ \text{out} = \left\lfloor \frac{\text{in} + 2\cdot\text{padding} - \text{kernel}}{\text{stride}} \right\rfloor + 1 \]

Con **Pooling** 2×2 e stride 2 spesso **dimezzi** H e W.  
Con **Flatten** converti `(N, C, H, W)` in `(N, C·H·W)` prima di un `nn.Linear`.

### Suggerimenti pratici
- Stampa le **shape** in `forward` quando qualcosa non torna.  
- Parti **semplice**; aggiungi funzioni (dropout/BN) una alla volta.  
- Controlla la **normalizzazione** degli input; i modelli pre-addestrati si aspettano medie/std specifiche.  
- Valida sempre su un **holdout** e monitora **loss e accuratezza**.



## 8. Esercizi

1. **Gioca coi kernel:** modifica il kernel nella demo di convoluzione (blur, sharpen, ecc.) e osserva le differenze.  
2. **CNN più profonda:** aggiungi un terzo layer conv; adegua l'input del `Linear`.  
3. **Sweep di regolarizzazione:** prova diversi `p` del Dropout e valori di `weight_decay`.  
4. **BatchNorm on/off:** confronta le curve di apprendimento con e senza BN.  
5. **Scheduler LR:** sostituisci `StepLR` con `OneCycleLR` o `CosineAnnealingLR`.  
6. **Early stopping:** implementa uno stop precoce sulla loss di validazione.  
7. **Confusion matrix:** calcola e visualizza una matrice di confusione per MNIST.  
8. **Transfer learning:** sblocca l’ultimo blocco residuo in ResNet18 e fai fine-tuning.
